In [ ]:
'''模組'''
import lupa
import re
import random
import numpy as np
import pandas as pd
import itertools
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import seaborn as sns
import datetime
import ast
from collections import Counter #Orderdict
from statistics import mean
from scipy import stats
import glob  #glob is used to return all file paths that match a specific pattern

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

sns.set(style='white',font_scale= 2.0)
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']#轉換成中文字體
plt.rcParams['axes.unicode_minus'] = False

pd.set_option('float_format','{:1.2f}'.format)

pd.options.display.max_rows = 300
pd.options.display.min_rows = 300

# 整體殺率實驗

In [ ]:
'''讀取最新指定訊息'''
# max_file = r'D:\GitLab\DataSinence\Log\2_bairen_bigR_water_in_1\baccarat_3_percent_killrate.log'

# df = pd.read_csv(max_file,
#              sep=" ",
#             header = None)
'''----------------------------------調整參數----------------------------------'''
file_type_ = r'\game_test_adjusted_cfg_2.log'
base_water_line = 3000000
file_path = fr'C:\Users\user\Dropbox\shared_company_folder\lianhuan_duobao_WR_pro_edited_ian'
'''--------------------------讀取指定檔案的文件並合併-----------------------------'''    
max_file = file_path
file_type = file_type_
files = glob.glob(max_file + file_type)
df_list = []
for i in files:
    df = pd.read_csv(i,
                 sep=" ",
                header = None)
    df_list.append(df)
df = pd.concat(df_list,axis=0,ignore_index = True)

'''計算'''
df["new"] = ''
for i in range(len(df.columns)-1):
    df["new"] += df[i]
list_ = []
for i in range(len(df)):
    try:
        list_.append(ast.literal_eval(df['new'][i]))
    except:
        df["new"][i] = df["new"][i-1]

columns = list(ast.literal_eval(df['new'][i]).keys()) #取出column值
new_df = pd.DataFrame(columns = columns,
                     data = list_)


'''---------------------------------輸贏局數-----------------------------------''' 
a = new_df[new_df.flow_money != 0].reset_index(drop=True)           #排除震盪那一段
a['in_out'].sort_values()#.to_frame()
# print('',new_df['total_rwd'].value_counts())

'''計算並輸出資料'''
#     if re.findall('lianhuan_duobao',max_file) != []:  #針對連環奪寶計算每局是輸還是贏
a_money_change = pd.json_normalize([ast.literal_eval(a.ext_data[i]) for i in range(len(a))])
a_money_change = pd.concat([a_money_change,a['in_out']],axis=1)
a.loc[:,('in_out')] = a_money_change['cache_turn_currency_change'] + a_money_change['in_out']
a.loc[:,('valid_score')] = a.loc[:,('bet_money')]
print('平均賠率：',a_money_change.rwd_times[a_money_change.rwd_times.notna()].mean())
zero_ = 0
for i in range(len(a)):
    if a_money_change['note'][i] == abs(a_money_change['cache_turn_currency_change'][i]):
        zero_ +=1
print('中獎率: {:1.3f}'.format(1-(zero_/len(a))))
lose_count = len(a[a['in_out'] < 0])
win_count = len(a[a['in_out'] > 0])
win_total = sum(a[a['in_out'] > 0]['in_out'])
lose_total = sum(a[a['in_out'] < 0]['in_out'])
print('輸牌局數：',lose_count)
print('贏牌局數：',win_count)
print('總局數：',len(a))
print('輸贏局數比: {:1,.2f}'.format(lose_count/win_count),'比 1')
print('總贏分: ${:1,.0f}'.format(win_total))
print('總輸分: ${:1,.0f}'.format(lose_total))
print('總輸贏: ${:1,.0f}'.format(sum(a['in_out'])))
print('總打碼: {:1,.0f}'.format(sum(abs(a['valid_score']))))
print('殺率: {:1,.2f}%'.format(-a['in_out'].sum()/a.valid_score.sum()*100))
print('贏分每局金額: ${:1,.0f}'.format(win_total/win_count))
print('輸分每局金額: ${:1,.0f}'.format(lose_total/lose_count))
print('水線集中：',round(new_df['water_level'].mean()))
print('水線範圍(最高與最低的差距)：{:1,.0f}'.format(new_df['water_level'].nlargest(1).values[0]-new_df['water_level'].nsmallest(1).values[0]))

#     '''------------------------------水線走勢圖-----------------------------------'''
plt.figure(figsize = (20,8), dpi = 80)
plt.plot(a.water_level,linewidth = 1.5, alpha = 0.4)
plt.axhline(y =a.water_level.nlargest(1).values, c = 'orange', ls = '-.')
plt.axhline(y =a.water_level.nsmallest(1).values, c = 'orange', ls = '-.')
plt.axhline(y =base_water_line, c = 'red', ls = '--')
plt.tick_params(which = "both", labelsize = 25)

plt.yticks([a.water_level.nsmallest(1).values[0],base_water_line,a.water_level.nlargest(1).values[0]])
plt.gca().yaxis.set_major_formatter(lambda x,pos : '{:1.0f}K'.format(x*1e-3))

#     if re.findall('bigsmall',max_file) != []:    
#         print('豹子數：', len(new_df[(new_df['player_id'] == 100259)&(new_df['the_same'] == 1)]))
#         print('獲利豹子數：', len(new_df[(new_df['player_id'] == 100259)&(new_df['the_same'] == 1)&((new_df['in_out'] > 1))]))

# 各區間回至基準水位線的數值

In [ ]:
'''=================================================================================================================='''
'''================================================連環奪寶水線程式碼設定================================================'''
'''=================================================================================================================='''
'''-----------------------------------調整參數-----------------------------------'''
start_point = 1000000#int(input('請輸入起始水位：'))
end_point = 1700000 #int(input('請輸入結束水位：'))
standard_level = 30000000
files = r'C:\DataSinence_\Log\1_lianhuan_duobao_givein_WLV_count\game_test_random_givup.log'
'''----------------------------------讀取指定檔案的文件並合併----------------------------------'''
df = pd.read_csv(files,
             sep=" ",
            header = None)

df['new'] = ''
for i in range(len(df.columns)-1):
    df['new'] += df[i]

'''計算'''
list_ = []
for i in range(len(df)):
    list_.append(ast.literal_eval(df['new'][i]))

columns = list(ast.literal_eval(df['new'][i]).keys()) #取出column值
new_df = pd.DataFrame(columns = columns,
                     data = list_)

if re.findall('bigsmall',max_file[0]) != []:  #對於bigsmall做特殊操作，因為有返還問題，有機會有連續兩局水線是一樣的。
    count = 0
    for i in range(len(new_df)):
        count+=1
        if count>len(new_df):
            break
        elif new_df.water_level[i] == new_df.water_level[i+1]:
            new_df.drop(index = i, axis=0, inplace= True) 
new_df.reset_index(inplace=True,drop = True)


'''----------------------------------取出該區間實驗各值----------------------------------'''
list_water_line = []
count_ = 0
for i in new_df.water_level:
    if (count_ == 0) and (i < end_point):
        list_water_line.insert(0,[])
        list_water_line[0].append(i)
        count_ += 1
    elif i >= end_point:
        list_water_line[0].append(i)
        count_ = 0
    else:
        list_water_line[0].append(i)
        count_ += 1

list_water_line = pd.DataFrame(list_water_line).transpose()
list_water_line['total'] = list_water_line.mean(axis=1)

a = new_df.reset_index(drop=True)          #這裡可以調整擷取數值
a['in_out'].sort_values()#.to_frame()
# print('',new_df['total_rwd'].value_counts())
if re.findall('lianhuan_duobao',files) != []:  #針對連環奪寶計算每局是輸還是贏
    a_money_change = pd.json_normalize([ast.literal_eval(new_df.ext_data[i]) for i in range(len(new_df))])
    a_money_change = pd.concat([a_money_change,a['in_out']],axis=1)
    a.loc[:,('in_out')] = a_money_change['cache_turn_currency_change'] + a_money_change['in_out']
    a.loc[:,('valid_score')] = a.loc[:,('bet_money')]
lose_count = len(a[a['in_out'] <= 0])
win_count = len(a[a['in_out'] > 0])
win_total = sum(a[a['in_out'] > 0]['in_out'])
lose_total = sum(a[a['in_out'] <= 0]['in_out'])
round_count = int(list_water_line.count().mean())
moving_num_per_round = start_point-end_point/round_count
print('殺率: {:1,.2f}%'.format(-a['in_out'].sum()/new_df.valid_score.sum()*100))
print("起始水位:", start_point)
print('平均所需局數：', round_count)
# print('總投注量：{:1,.0f}'.format(new_df.valid_score[0]*list_water_line.count().mean()))
print('總投注量：{:1,.0f}'.format(new_df.valid_score.sum()/10))

if re.findall('givout',files) != []:   #判斷是收水還是放水
    print('平均每局下降(斜率)：{:1,.0f}'.format(moving_num_per_round))
else:
    print('平均每局上升(斜率)：{:1,.0f}'.format(moving_num_per_round))

# print('平均每局下降金額與押注金額比：{:1,.2f}'.format(moving_num_per_round/new_df.valid_score[0]))


'''----------------------------------各房間水線參數&並設定範圍線----------------------------------'''
str_room_type_1 = '''{
[1] = {pool_range = {-9999999, 0.5},    fix_rate_lst = {[1] = 1, [2] = 1.24, [3] = 1.24, [4] = 1.02, [5] = 1.02, [6] = 1.02, [7] = 0.72, [8] = 0.72, [9] = 0.72, [10] = 0.24, [11] = 0.09, [12] = 0.09, [13] = 0.05, [14] = 0, [15] = 0, [16] = 0}},
[2] = {pool_range = {0.5, 0.7},    fix_rate_lst = {[1] = 1, [2] = 1.18, [3] = 1.18, [4] = 1.18, [5] = 1.03, [6] = 0.79, [7] = 0.79, [8] = 0.74, [9] = 0.74, [10] = 0.74, [11] = 0.74, [12] = 0.74, [13] = 0.19, [14] = 0.1, [15] = 0, [16] = 0}},
[3] = {pool_range = {0.7, 0.9},    fix_rate_lst = {[1] = 1, [2] = 1, [3] = 1, [4] = 1, [5] = 1, [6] = 1, [7] = 1, [8] = 1, [9] = 1, [10] = 1, [11] = 1, [12] = 1, [13] = 0.8, [14] = 0.5, [15] = 0.2, [16] = 0}},
[4] = {pool_range = {0.9, 1},    fix_rate_lst = {[1] = 1, [2] = 1, [3] = 1, [4] = 1, [5] = 1, [6] = 1, [7] = 1, [8] = 1, [9] = 1, [10] = 1, [11] = 1, [12] = 1, [13] = 0.9, [14] = 0.9, [15] = 0.5, [16] = 0.5}},
[5] = {pool_range = {1, 1.1},    fix_rate_lst = {[1] = 1, [2] = 0.98, [3] = 0.98, [4] = 0.98, [5] = 0.98, [6] = 0.98, [7] = 0.98, [8] = 1.43, [9] = 1.43, [10] = 1.43, [11] = 1.43, [12] = 1.09, [13] = 1.09, [14] = 1.09, [15] = 1.09, [16] = 1.09}},
[6] = {pool_range = {1.1, 1.3},    fix_rate_lst = {[1] = 1, [2] = 0.97, [3] = 0.97, [4] = 0.97, [5] = 0.97, [6] = 0.97, [7] = 0.97, [8] = 1.87, [9] = 1.8, [10] = 1.74, [11] = 1.38, [12] = 1.29, [13] = 1.29, [14] = 1.19, [15] = 1.19, [16] = 1.19}},
[7] = {pool_range = {1.3, 1.5},    fix_rate_lst = {[1] = 1, [2] = 0.97, [3] = 0.97, [4] = 0.97, [5] = 0.97, [6] = 0.97, [7] = 0.97, [8] = 2.13, [9] = 2.13, [10] = 1.94, [11] = 1.46, [12] = 1.46, [13] = 1.46, [14] = 1.26, [15] = 1.26, [16] = 1.26}},
[8] = {pool_range = {1.5, 99999999},    fix_rate_lst = {[1] = 1, [2] = 0.96, [3] = 0.96, [4] = 0.96, [5] = 0.96, [6] = 0.96, [7] = 0.96, [8] = 2.4, [9] = 2.4, [10] = 2.12, [11] = 1.73, [12] = 1.73, [13] = 1.73, [14] = 1.35, [15] = 1.35, [16] = 1.35}},
}
''' 
lua = lupa.LuaRuntime()    #解讀資料
Fee_Pool_dict_ = {}
for i,x in lua.eval(str_room_type_1).items():
    list_ = []
    for a,b in x.items():
        list_.append((a,b))
        Fee_Pool_dict_[i] = dict(list_)
for i in range(1,9):
    list_ = []
    for y,x in Fee_Pool_dict_[i]['pool_range'].items():
        list_.append(x*standard_level)
    Fee_Pool_dict_[i]['pool_range'] = list_

highest_number = 0   
for i in new_df['water_level']:
    if i > highest_number:
        highest_number = i
        
lowest_number = highest_number 
for i in new_df['water_level']:
        if i <lowest_number :
            lowest_number = i
'''----------------------------------利用設定的範圍線劃出各圖----------------------------------'''
for_ticks = []
for i in range(1,9):
    for_ticks.append(Fee_Pool_dict_[i]['pool_range'][1])
    
for_ticks.append(int(for_ticks[-2]*1.5)) #設放水最長的位置
for_ticks.insert(0 ,int(for_ticks[0]/1.2)) #設收水最長的位置

plt.figure(figsize = (16,8))
plt.plot(list_water_line['total'],linewidth = 1.5)
for i in for_ticks:
    if i == standard_level:
        plt.axhline(y=i, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-.')
    else:    
        plt.axhline(y=i, xmin=0, xmax=3, c="green", linewidth=1, zorder=0, ls = '--', alpha = 0.5)

# plt.axhline(y=highest_number, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-')
# plt.axhline(y=lowest_number, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-')

plt.tick_params(which = "both", labelsize = 25)
plt.gca().yaxis.set_major_formatter(lambda x,pos : '{:1.0f}M'.format(x*1e-6))
# plt.yticks([i for i in for_ticks if i != 300000000]);
# plt.ylim([600000000,670000000])
# plt.xticks([15643,len(new_df)])
#plt.title(f'_fee_percent:{_fee_percent}，fee_percent:{fee_percent}，平均殺率：{round(np.mean(kill_rate)*100,3)}%' , fontsize = 20)


# plt.figure(figsize = (16,8))
list_water_line.drop('total',axis = 1, inplace=True)
for i in range(len(list_water_line.columns)):
    plt.plot(list_water_line[i],linewidth = 1.5, alpha = 0.4)
'''------------------------------------斜率線(y = a-b*x,b為每局下降數)--------------------------------------------''' 
x = np.linspace(1,round_count,round_count)
y = start_point-(moving_num_per_round*x)
plt.plot(x,y,color = 'black')

for i in for_ticks:
    if i == standard_level:
        plt.axhline(y=i, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-.')
    else:    
        plt.axhline(y=i, xmin=0, xmax=3, c="green", linewidth=1, zorder=0, ls = '--', alpha = 0.5)
# for i in for_ticks
plt.axhline(y=start_point, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-.')
plt.axhline(y=end_point, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-.')
# plt.axhline(y=highest_number, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-')
# plt.axhline(y=lowest_number, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-')

plt.tick_params(which = "both", labelsize = 25)
plt.gca().yaxis.set_major_formatter(lambda x,pos : '{:1.0f}K'.format(x*1e-3))
(plt.yticks([i for i in for_ticks if i>=standard_level]) if end_point < start_point else plt.yticks([i for i in for_ticks if i<=standard_level]))
(plt.ylim([end_point,start_point]) if end_point < start_point else plt.ylim([start_point, end_point]))

# 水線級距

In [ ]:
'''---------------------------------------調整參數----------------------------------------'''
water_base_level = 192000000
folder_path = r'D:\GitLab\DataSinence\Log\4_south_nn_bigR'
file_type = r'\game_test.log'
files = glob.glob(folder_path + file_type)
'''----------------------------------讀取指定檔案的文件並合併----------------------------------'''
df_list = []
for i in files:
    df = pd.read_csv(i,
                 sep=" ",
                header = None)
    df_list.append(df)
df = pd.concat(df_list,axis=0,ignore_index = True)



df['new'] = ''
for i in range(len(df.columns)-1):
    df['new'] += df[i]

'''計算'''
list_ = []
for i in range(len(df)):
    list_.append(ast.literal_eval(df['new'][i]))

columns = list(ast.literal_eval(df['new'][i]).keys()) #取出column值
new_df = pd.DataFrame(columns = columns,
                     data = list_)

'''-------------------------------------------------各房間水線參數-------------------------------------------------'''
str_room_type_1 = '''{
		[1] = { desc = "收水3级", target = "AI", diff_min = -99999999999, diff_max = -100000000, prob = 10000 },
		[2] = { desc = "收水2级", target = "AI", diff_min = -100000000, diff_max = -50000000, prob = 8000 },
		[3] = { desc = "收水1级", target = "AI", diff_min = -50000000, diff_max = -36800000, prob = 6000 },
		[4] = { desc = "收水0级", target = "AI", diff_min = -36800000, diff_max = 0, prob = 2000 },
		[5] = { desc = "放水1级", target = "PL", diff_min = 4680000, diff_max = 30000000, prob = 5000 },
		[6] = { desc = "放水2级", target = "PL", diff_min = 30000000, diff_max = 500000000, prob = 2000 },
		[7] = { desc = "放水3级", target = "PL", diff_min = 500000000, diff_max = 99999999999, prob = 1000 },
    }
''' 
str_room_type_2 = '''{
		[1] = { desc = "收水3级", target = "AI", diff_min = -99999999999, diff_max = -100000000, prob = 10000 },
		[2] = { desc = "收水2级", target = "AI", diff_min = -100000000, diff_max = -50000000, prob = 8000 },
		[3] = { desc = "收水1级", target = "AI", diff_min = -50000000, diff_max = -20000000, prob = 5500 },
		[4] = { desc = "收水0级", target = "AI", diff_min = -18800000, diff_max = 0, prob = 4300 },
		[5] = { desc = "放水1级", target = "PL", diff_min = 0, diff_max = 35880000, prob = 3800 },
		[6] = { desc = "放水2级", target = "PL", diff_min = 35880000, diff_max = 126800000, prob = 3500 },
		[7] = { desc = "放水3级", target = "PL", diff_min = 126800000, diff_max = 99999999999, prob = 3000 },
     }
''' 
lua = lupa.LuaRuntime()
Fee_Pool_dict_ = {}
if new_df.room_type[0] == 1:   #for loop 取正確的房間參數
    for i,x in lua.eval(str_room_type_1).items():
        list_ = []
        for a,b in x.items():
            list_.append((a,b))
            Fee_Pool_dict_[i] = dict(list_)
else:
    if str_room_type_2 is not None:
        for i,x in lua.eval(str_room_type_2).items():
            list_ = []
            for a,b in x.items():
                list_.append((a,b))
                Fee_Pool_dict_[i] = dict(list_)

'''-----------------------------------------------------取級距最大最小值------------------------------------------------'''
highest_number = 0
for i in new_df['water_level']:
    if i > highest_number:
        highest_number = i
        
lowest_number = highest_number 
for i in new_df['water_level']:
        if i <lowest_number :
            lowest_number = i

'''-------------------------------------------------------水線範圍-----------------------------------------------------'''
for_ticks = []
for i in range(1,7):
    for_ticks.append(new_df.water_level[0] + Fee_Pool_dict_[i]['diff_max'])
# for_ticks[-1] = for_ticks[-2]*1.5
# for_ticks.insert(0 ,int(for_ticks[0]/1.2)) #設收水最長的位置

plt.figure(figsize = (20,8), dpi = 80)
for i in range(len(new_df)):
    plt.plot(new_df.iloc[1*i*10000:1*i*10000+10000,:].water_level,linewidth = 1.5, alpha = 0.4)
# plt.plot(new_df.water_level.,linewidth = 1.5, alpha = 0.4)
# for i in for_ticks:
#     plt.axhline(y=i, xmin=0, xmax=3, c="green", linewidth=1, zorder=0, ls = '--')
plt.axhline(y=water_base_level, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-.')
plt.axhline(y=highest_number, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-')
plt.axhline(y=lowest_number, xmin=0, xmax=3, c="red", linewidth=1, zorder=0, ls = '-')
plt.yticks([highest_number,water_base_level,lowest_number])
plt.tick_params(which = "both", labelsize = 25)
plt.gca().yaxis.set_major_formatter(lambda x,pos : '{:1.0f}K'.format(x*1e-3))
# plt.yticks(for_ticks);
# plt.xticks([15643,len(new_df)])
#plt.title(f'_fee_percent:{_fee_percent}，fee_percent:{fee_percent}，平均殺率：{round(np.mean(kill_rate)*100,3)}%' , fontsize = 20)

'''------------------------------------------------------輸出資料----------------------------------------------------'''
a = new_df[::].reset_index(drop=True)  #調整擷取數值
a['in_out'].sort_values()#.to_frame()
if re.findall('lianhuan_duobao',folder_path) != []:  #針對連環奪寶計算每局是輸還是贏
    a_money_change = pd.json_normalize([ast.literal_eval(new_df.ext_data[i]) for i in range(len(new_df))])
    a_money_change = pd.concat([a_money_change,a['in_out']],axis=1)
    a.loc[:,('in_out')] = a_money_change['cache_turn_currency_change'] + a_money_change['in_out']
    a.loc[:,('valid_score')] = a.loc[:,('bet_money')]
lose_count = len(a[a['in_out'] <= 0])
win_count = len(a[a['in_out'] > 0])
win_total = sum(a[a['in_out'] > 0]['in_out'])
lose_total = sum(a[a['in_out'] <= 0]['in_out'])
print('輸牌局數',lose_count)
print('贏牌局數',win_count)
print('總局數',len(a))
print('輸贏局數比: {:1,.2f}'.format(len(a[a['in_out'] < 0]['in_out'])/len(a[a['in_out'] >= 0]['in_out'])),'比 1')
print('總贏分: ${:1,.0f}'.format(win_total))
print('總輸分: ${:1,.0f}'.format(lose_total))
print('總輸贏: ${:1,.0f}'.format(sum(a['in_out'])))
print('流水: {:1,.0f}'.format(sum(abs(a['in_out']))))
print('殺率: {:1,.2f}%'.format(-sum(a['in_out'])/a.valid_score.sum()*100))
# print('贏分每局金額: ${:1,.0f}'.format(win_total/win_count))
print('輸分每局金額: ${:1,.0f}'.format(lose_total/lose_count))
water_range_difference = new_df.water_level.sort_values(ascending=False).head(1).values-new_df.water_level.sort_values().head(1).values#-new_df.water_level.nsmallest().head(1)
print('水線波動最大: {:1,.0f}'.format(water_range_difference[0]))
num_high = highest_number-water_base_level
num_low = water_base_level-lowest_number
high_rate = round(num_high/new_df.valid_score.mean(),1)
low_rate = round(num_low/new_df.valid_score.mean(),1)

print(f'最大波動為投注金的{high_rate},最小波動為投注金的{low_rate}' if num_high > num_low else f'最大波動為投注金的{low_rate},最小波動為投注金的{high_rate}')
print('------------------------------ˇ-----------------------------')
'''----------------------------------------------------常態分佈圖---------------------------------------------------'''
plt.figure(figsize = (20,8), dpi = 80)
sns.histplot(new_df.water_level)
m = new_df['water_level'].mean()                  # X_bar 的平均
med = new_df['water_level'].median()  
se = new_df['water_level'].std()                  # 標準誤
ci = stats.norm.interval(0.95, m, se)
print('標準差: {:1,.0f}'.format(se)) 
print('中位數: {:1,.0f}'.format(med)) 
print('平均值: {:1,.0f}'.format(m))
print('水線值分布圖信賴區間為: ',ci)

plt.axvline(m, color='green', linestyle='dashed', linewidth=2)
plt.axvline(med, color='red', linestyle='dashed', linewidth=2)
plt.axvline(ci[0], color='magenta', linestyle='dashed', linewidth=2)
plt.axvline(ci[1], color='magenta', linestyle='dashed', linewidth=2)
plt.gca().xaxis.set_major_formatter(lambda x,pos : '{:1.1f}M'.format(x*1e-6))
# re.findall('waterrange_(\D+)',max_file)
plt.legend([''])

# 期望殺率與投注金的關係

In [ ]:
'''讀取最merge的文件'''
import glob  #glob is used to return all file paths that match a specific pattern
import os.path
'''-------------------------------------------調整參數------------------------------------------'''
path_text = fr'D:\GitLab\DataSinence\Log\2_north_nn_killrate_VS_betamount'  #路徑
bet_amount = 10000   #每次幾回
file_type = r'\game_test.log'   #log的名稱
'''------------------------------------讀取指定檔案的文件並合併-------------------------------------'''
for file in [1,100]: 
    folder_path = path_text
    file_type = file_type
    files = glob.glob(folder_path + f'{file}W'+file_type)
    if files == []:
        folder_path = path_text
        file_type = r'\*.log'
        files = glob.glob(folder_path +  f'{file}W'+ file_type)   
        df_list = []
        for i in files:
            df = pd.read_csv(i,
                         sep=" ",
                        header = None)
            df_list.append(df)
        df = pd.concat(df_list,axis=0,ignore_index = True)       
    else:
        df = pd.read_csv(files[0], sep=' ', header = None)
        
    '''計算'''
    df['new'] = ''
    for i in range(len(df.columns)-1):
        df['new'] += df[i]
    '''jason格式轉成pandas'''
    list_ = []
    for i in range(len(df)):
        list_.append(ast.literal_eval(df['new'][i]))
        
    columns = list(ast.literal_eval(df['new'][i]).keys()) #取出column值
    new_df = pd.DataFrame(columns = columns,
                         data = list_)
    
    if re.findall('lianhuan_duobao',folder_path) != []:  #針對連環奪寶計算每局是輸還是贏
        a_money_change = pd.json_normalize([ast.literal_eval(new_df.ext_data[i]) for i in range(len(new_df))])
        a_money_change = pd.concat([a_money_change,new_df['in_out']],axis=1)
        new_df.loc[:,'in_out'] = a_money_change['cache_turn_currency_change'] + a_money_change['in_out']
        new_df.loc[:,'valid_score'] = new_df.loc[:,'bet_money']
        
    '''-------------------------------------------計算殺率線圖-------------------------------------------'''
    kill_rate_average = []
    round_ = int(np.ceil(len(new_df)/bet_amount)) #共幾回
    for i in range(1,round_+1):
        in_out_cumsum = new_df['in_out'][bet_amount*i:(i+1)*bet_amount].cumsum()   #累計玩家每1萬筆總輸贏
        
        flow_money_cumsum = new_df['valid_score'][bet_amount*i:(i+1)*bet_amount].cumsum()  #每柱流水計算
        # '''總打碼(流水加總 - 平台輸贏)'''
        # new_df['bet_cumsum'] = new_df['flow_money_cumsum'] + new_df['in_out_cumsum']
        
        kill_rate_ = -(in_out_cumsum / flow_money_cumsum)*100  # calculate kill_rate
        kill_rate_average.append(kill_rate_.values)
        
#     kill_rate_average = pd.concat(kill_rate_average,axis=1,keys = [i for i in range(len(kill_rate_average))])
    kill_rate_total = pd.DataFrame(kill_rate_average).transpose().mean(axis=1)
    

    '''---------------------------------------------------輸出資料-------------------------------------------------'''
    a = new_df[::].reset_index(drop=True)  #調整擷取數值
    
    lose_count = len(a[a['in_out'] <= 0])
    win_count = len(a[a['in_out'] > 0])
    win_total = sum(a[a['in_out'] > 0]['in_out'])
    lose_total = sum(a[a['in_out'] <= 0]['in_out'])
    kill_rate = -a['in_out'].sum()/a.valid_score.sum()*100
    
    print('總局數',len(a))
    print('輸贏局數比: {:1,.2f}'.format(len(a[a['in_out'] < 0]['in_out'])/len(a[a['in_out'] >= 0]['in_out'])),'比 1')
    print('總輸贏: ${:1,.0f}'.format(sum(a['in_out'])))
    print('總打碼: {:1,.0f}'.format(new_df.valid_score.sum()/round_))
    print('殺率: {:1,.2f}%'.format(kill_rate))
    print()

    kill_rate_range = [2.5,3.5]
    '''------------------------------正常畫圖-----------------------------------'''
    plt.figure(figsize = (18,6))
    [plt.plot(kill_rate_average[i][:10000]) for i in range(len(kill_rate_average))]
    kill_rate_range = kill_rate_range   #for yticks & axline
    plt.ylim(-3,10)
    plt.gca().yaxis.set_major_formatter(lambda x,pos : '{:1.1f}%'.format(x))
    plt.axhline(y=kill_rate_range[1], xmin=0, xmax=3, c="red", linewidth=2, zorder=0, ls = '-.')
    plt.axhline(y=kill_rate_range[0], xmin=0, xmax=3, c="red", linewidth=2, zorder=0, ls = '-.')
    plt.axhline(y=kill_rate, xmin=0, xmax=3, c="green", linewidth=1, zorder=0, ls = '--')
    plt.tick_params(which = "both", labelsize = 25)
#     plt.yticks([kill_rate_range[0],kill_rate,kill_rate_range[1]])
#     '''------------------------------平均畫圖-----------------------------------'''
#     plt.figure(figsize = (18,6))
#     kill_rate_total.plot()
#     kill_rate_range = kill_rate_range  #for yticks & axline
#     plt.ylim(-3,10)
#     plt.gca().yaxis.set_major_formatter(lambda x,pos : '{:1.1f}%'.format(x))
#     plt.axhline(y=kill_rate_range[1], xmin=0, xmax=3, c="red", linewidth=2, zorder=0, ls = '-.')
#     plt.axhline(y=kill_rate_range[0], xmin=0, xmax=3, c="red", linewidth=2, zorder=0, ls = '-.')
#     plt.axhline(y=kill_rate, xmin=0, xmax=3, c="green", linewidth=1, zorder=0, ls = '--')
#     plt.tick_params(which = "both", labelsize = 25)
# #     plt.yticks([kill_rate_range[0],kill_rate,kill_rate_range[1]])    
    
#     '''------------------------------每50個取平均畫圖-----------------------------------'''
# #     num = []
# #     for i in range(0,len(kill_rate_total),100):
# #         num.append(np.mean(kill_rate_total[i:i+99]))
#     plt.figure(figsize = (18,6))
#     plt.plot(kill_rate_total.rolling(window = 50).mean())   #ex股票20日線
#     plt.ylim(-3,10)
#     plt.gca().yaxis.set_major_formatter(lambda x,pos : '{:1.1f}%'.format(x))
#     plt.axhline(y=kill_rate_range[0], xmin=0, xmax=3, c="red", linewidth=2, zorder=0, ls = '-.')
#     plt.axhline(y=kill_rate_range[1], xmin=0, xmax=3, c="red", linewidth=2, zorder=0, ls = '-.')
#     plt.axhline(y=kill_rate, xmin=0, xmax=3, c="green", linewidth=1, zorder=0, ls = '--')
#     plt.tick_params(which = "both", labelsize = 25)
# #     plt.yticks([kill_rate_range[0],kill_rate_range[1]])
#     tick_length = kill_rate_total.rolling(window = 50).mean()[kill_rate_total.rolling(window = 50).mean().notna()]
# #     plt.xticks([i for i in range(10000-len(tick_length),10000,2000)][]);
    